# Life-Cycle Labor Supply of Couples

Solves and simulates a $T$-period labor supply model with two-earner couples. <br>
**Motivated** by the study “Are Marriage-Related Taxes and Social Security Benefits Holding Back Female Labor Supply?” by Borella et al. (forthcoming). <br>
**Goal** is to replicate effects of individual vs. joint taxation.

For simplicity, couples cannot divorce nor save.

The **Bellman equation** and the recursive formulation of our simple model is 
$$
\begin{align*}
V_{t}(K_{1,t},K_{2,t}) & =\max_{h_{1,t},h_{2,t}}U(c_{t},h_{1,t},h_{2,t})+\beta V_{t+1}(K_{1,t+1},K_{2,t+1})\\
c_{t} & =\sum_{j=1}^{2}w_{j,t}h_{j,t}-T(w_{1,t}h_{1,t},w_{2,t}h_{2,t})\\
\log w_{j,t} & =\alpha_{j,0}+\alpha_{j,1}K_{j,t},\;j\in\{1,2\}\\
K_{j,t+1} & =(1-\delta)K_{j,t}+h_{j,t},\;j\in\{1,2\}
\end{align*}
$$

**Preferences** are sum of individuals
$$
U(c_{t},h_{1,t},h_{2,t})=2\frac{(c_{t}/2)^{1+\eta}}{1+\eta}-\rho_{1}\frac{h_{1,t}^{1+\gamma}}{1+\gamma}-\rho_{2}\frac{h_{2,t}^{1+\gamma}}{1+\gamma}
$$

**Taxes** are on the household level
$$
T(Y_{1},Y_{2})=(1-\lambda(Y_{1}+Y_{2})^{-\tau})\cdot(Y_{1}+Y_{2})
$$

**Terminal period:** There are no bequests such that
$$
V_{T}(K_{1,T},K_{2,T})  =\max_{h_{1,T},h_{2,T}}U(c_{T},h_{1,T},h_{2,T})
$$

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numba as nb
import matplotlib.pyplot as plt

# Consumption-Saving Model

In [2]:
# load local model file and initialize model class
from DynHouseholdLaborModel import DynHouseholdLaborModelClass
model = DynHouseholdLaborModelClass()

par = model.par
sol = model.sol
sim = model.sim

## Estimation.
Imagine that labor supply in the data is 0.7 for member 1 and 0.65 for member 2 in the first period (say, age 25).

In [3]:
theta_names = ('rho_1','rho_2') # names (list) of parameters to estimate
theta = np.array([0.04,0.03]) # parameter vector (array)
data_moments = np.array([0.7,0.65]) # moments to match (from Statistics Denmark, say)

def obj_func(theta,theta_names,data_moments,model_in):
    model = model_in.copy() # safety measure

    # set the current parameters
    for p,par_name in enumerate(theta_names):
        setattr(model.par, par_name, theta[p])
        print(f'{par_name}:{theta[p]:2.3f} ',end='')

    # solve the model for these parameters
    model.solve()

    # simulate the model using this solution
    model.simulate()

    # calculate the relevant moments
    h1 = np.mean(model.sim.h1[:,0])
    h2 = np.mean(model.sim.h2[:,0])
    sim_moments = np.array([h1,h2])

    # return the the objective function
    W = np.eye(2)
    diff = data_moments - sim_moments
    obj = diff.T @ W @ diff

    print(f'-> obj = {obj:2.7f}')
    return obj
    
# test objective function    
obj = obj_func(theta,theta_names,data_moments,model)

rho_1:0.040 rho_2:0.030 -> obj = 0.0353346


In [4]:
# call an optimizer
from scipy.optimize import minimize

init_theta = np.array([0.05,0.05])
res = minimize(obj_func,init_theta,args=(theta_names,data_moments,model),method='nelder-mead')

rho_1:0.050 rho_2:0.050 -> obj = 0.0032287
rho_1:0.053 rho_2:0.050 -> obj = 0.0035925
rho_1:0.050 rho_2:0.053 -> obj = 0.0020226
rho_1:0.048 rho_2:0.053 -> obj = 0.0021687
rho_1:0.048 rho_2:0.055 -> obj = 0.0015151
rho_1:0.046 rho_2:0.058 -> obj = 0.0015704
rho_1:0.050 rho_2:0.055 -> obj = 0.0012030
rho_1:0.051 rho_2:0.056 -> obj = 0.0008633
rho_1:0.049 rho_2:0.059 -> obj = 0.0007679
rho_1:0.048 rho_2:0.062 -> obj = 0.0009409
rho_1:0.052 rho_2:0.060 -> obj = 0.0002334
rho_1:0.055 rho_2:0.062 -> obj = 0.0000629
rho_1:0.052 rho_2:0.065 -> obj = 0.0001544
rho_1:0.059 rho_2:0.069 -> obj = 0.0003107
rho_1:0.056 rho_2:0.066 -> obj = 0.0000535
rho_1:0.059 rho_2:0.064 -> obj = 0.0003838
rho_1:0.054 rho_2:0.065 -> obj = 0.0000169
rho_1:0.055 rho_2:0.068 -> obj = 0.0001994
rho_1:0.055 rho_2:0.064 -> obj = 0.0000074
rho_1:0.053 rho_2:0.062 -> obj = 0.0000688
rho_1:0.055 rho_2:0.065 -> obj = 0.0000126
rho_1:0.056 rho_2:0.065 -> obj = 0.0000583
rho_1:0.055 rho_2:0.065 -> obj = 0.0000019
rho_1:0.054

In [5]:
# estimated parameters
res.x

array([0.05472504, 0.06430852])